#Aim

Given a large set of sequences or graphs with ordered vertices find small vertex ordered subsequences that are most discriminative for the set.

Steps:
- devise a negative set
- learn a discriminative model
- annotate importance on vertices
- extract max subarrays 
- cluster them 
 - use fast EDeN string kernel 
 - custering algorithm
 
Output: 
1. all sequence motives in each cluster
2. all initial sequences with motif location (begin,end) and cluster id (build regex from all seqs in cluster and run a find iterator)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#code for making artificial dataset
import random
def random_string(length,alphabet_list):
    rand_str = ''.join(random.choice(alphabet_list) for i in range(length))
    return rand_str

def perturb(seed,alphabet_list,p=0.5):
    seq=''
    for c in seed:
        if random.random() < p: c = random.choice(alphabet_list)
        seq += c
    return seq

def make_artificial_dataset(alphabet='ACGU', motives=None, motif_length=6, sequence_length=100, n_sequences=1000, n_motives=2, p=0.2):
    alphabet_list=[c for c in alphabet]
    
    if motives is None:
        motives=[]
        for i in range(n_motives):
            motives.append(random_string(motif_length,alphabet_list))
    else:
        motif_length = len(motives[0])
        n_motives = len(motives)
        
    flanking_length = (sequence_length - motif_length ) / 2
    n_seq_per_motif = n_sequences / n_motives

    counter=0
    seqs=[]
    for i in range(n_seq_per_motif):
        for j in range(n_motives):
            left_flanking = random_string(flanking_length,alphabet_list)
            right_flanking = random_string(flanking_length,alphabet_list)
            noisy_motif = perturb(motives[j],alphabet_list,p)
            seq = left_flanking + noisy_motif + right_flanking
            seqs.append(('>ID%d'%counter,seq))
            counter += 1
    return motives, seqs

#Experimental Setup

In [3]:
#setup parameters
alphabet='ACGU'
motives=['AAAAAAAAAA','CCCCCCCCCC','GGGGGGGGGG','UUUUUUUUUU']
sequence_length=100
n_sequences=1000
p=0.3

#make dataset
motives, seqs = make_artificial_dataset(alphabet=alphabet,motives=motives,sequence_length=sequence_length,n_sequences=n_sequences,p=p)

#display
print 'Motives and sample of their perturbed variants:'
alphabet_list=[c for c in alphabet]
for motif in motives: 
    print
    print motif,
    for i in range(9):
        print perturb(motif,alphabet_list,p=p),

Motives and sample of their perturbed variants:

AAAAAAAAAA AGAAAACAAA AAAAAAAUAA AAGAUAAACA AAUGAAAAAA AAACAGAAAA AAAAAACAAA AAAAAAAAAA AAAAGAAAUA ACAAUAAAAA
CCCCCCCCCC CGCCCCCCUC CCCCCUCACC CCUCCCGACU CCCCCCCCCC ACCUCCACCC CGUCCCACCC CACCUCACCC AACCCCCCCC CCCCCCCCGC
GGGGGGGGGG GGGGGGGGGG GGGGGGGGUG GGAGGGGGUG GGGGCGAGGC GGGGGGGGUG GAGGACGGGU GGGGGGUUGG GUGCGUUGGG GGUGGGGGGU
UUUUUUUUUU AUUUAUUUUU UCUCUUUUUU UUUUUCUUGU UUUUUUGUUU GUUUUUUUGA AUUGUUUUGC UUUGUUCUAU GUUUCUGUUU UUCCUUUUUU


In [4]:
#save to file
fname='artificial_motif_search_dataset.fa'
with open(fname,'w') as f:
    for header,seq in seqs: 
        f.write(header+"\n")
        f.write(seq+"\n")

In [5]:
from eden.util import configure_logging
logger=configure_logging(verbosity=2)

In [7]:
%%time
from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=150, 
                       algorithm="Birch", 
                       threshold=0.1, 
                       n_clusters=4, 
                       branching_factor=50)
seqmot.fit(seqs)

model induction: 150 positive instances 25 secs
motives extraction: 649 motives 60 secs
motives clustering: 4 clusters 97 secs
after filtering: 432 motives 4 clusters 0 secs
motif model construction: 0 secs


CPU times: user 1min 27s, sys: 9.41 s, total: 1min 37s
Wall time: 3min 3s


In [7]:
for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

0
GGGGGGGGGG 22
GGGGGGGGG 4
UGGGGGGGGU 3
GGGGGGGGGC 3
GGGGGCGGGG 3
GGGCGGGGGG 3
CGGGGGGGU 3
AGGGGGGGGC 3
UGGGGGGGU 2
UGGGGGGGGA 2
UGAGGGGGGU 2
UGAGGGAGU 2
GGGUGGGGU 2
GGGGGUGGGG 2
GGGGAGGGGG 2
GGAGGGGGA 2
CGGGGGGGGU 2
CGGGGGGGA 2
UGGGUGGGGC 1
UGGGGUGGGG 1
UGGGGUGGG 1
UGGGGGGGA 1
UGGGGGGCGG 1
UGGGGGGCC 1
UGGGGGGAGG 1
UGGGGGGAGC 1
UGGGGGCGGG 1
UGGGGGAGGG 1
UGGGGCGGGC 1
UGGGGCGGGA 1
UGGGGAGCGC 1
UGGGAUAUA 1
UGCGGAGGU 1
UGAGUGUGA 1
UGAGUGGAGC 1
UGAGGGUGA 1
UGAGGGGGU 1
UGAGCGGGG 1
UGAGCGGAGA 1
UGAGCGCCCG 1
UCGGGCGGA 1
UAGGGGGGU 1
UAGAGGGAGU 1
UACGAGCGU 1
GUGACCCC 1
GGUGGCGAU 1
GGGUGACAC 1
GGGGUGGGA 1
GGGGGGGGGU 1
GGGGGGGGGA 1
GGGGGGGGC 1
GGGGGGGGA 1
GGGGGGCGGG 1
GGGGGGAGCG 1
GGGGGAGGGG 1
GGGGGAGGGC 1
GGGGGAGGG 1
GGGGCGGGGG 1
GGGGCGGGA 1
GGGGCCGCG 1
GGGGCCCCCU 1
GGGGAGGGGC 1
GGGGAGGGG 1
GGGGAGGGC 1
GGGCGGGAGG 1
GGGAGGGGGC 1
GGGAGGGGC 1
GGCGGUGGGG 1
GGCGGGGGC 1
GGCGGGGCGU 1
GGAGGGGGGU 1
GGAGGGGGGC 1
GCGGUGGGGC 1
GCGAGGGGU 1
GCGAGGGGGA 1
GCGAGGGCGG 1
GCCCGGGGC 1
GCCACGGGGU 1
GAGGGGGGGU 1
GAGGG

In [8]:
predictions=seqmot.predict(seqs, return_list=True)
for p in predictions: print p

[]
[]
[0, 3]
[]
[1]
[]
[0]
[0]
[1]
[0, 3]
[]
[2]
[0, 1]
[]
[0]
[]
[0, 1]
[1]
[0]
[2]
[1]
[3]
[0, 2]
[2]
[]
[2]
[0]
[2]
[]
[]
[0]
[]
[]
[3]
[0]
[]
[1]
[]
[1]
[2]
[]
[]
[]
[2]
[]
[1, 3]
[0, 1]
[]
[]
[]
[0]
[]
[1]
[0, 3]
[]
[2]
[1]
[3]
[]
[0]
[1]
[0]
[0]
[1, 2]
[1]
[3]
[0]
[2]
[1]
[3]
[0]
[]
[1]
[]
[0]
[2]
[]
[3]
[0]
[0, 2, 3]
[1, 3]
[3]
[0, 1]
[2]
[1]
[]
[]
[]
[]
[3]
[0]
[2]
[1]
[1, 3]
[0]
[2]
[1]
[3]
[0]
[]
[]
[3]
[]
[]
[]
[]
[]
[]
[1]
[3]
[]
[1, 2]
[1]
[0, 3]
[0]
[2]
[0, 1]
[3]
[]
[]
[]
[3]
[0]
[2]
[]
[3]
[0]
[]
[1]
[3]
[]
[]
[2]
[1, 3]
[]
[2]
[0]
[1, 3]
[0]
[0]
[1]
[3]
[0]
[]
[0, 1]
[3]
[0]
[2]
[]
[]
[0]
[]
[1]
[3]
[0]
[2]
[1]
[3]
[]
[2]
[1]
[1]
[0]
[2]
[]
[0]
[0]
[2]
[1]
[]
[0]
[2, 3]
[2]
[3]
[0]
[2]
[1]
[3]
[]
[]
[1]
[3]
[]
[2]
[1]
[3]
[0]
[]
[]
[3]
[0, 2]
[]
[1]
[1, 3]
[]
[2]
[1]
[]
[]
[]
[1]
[]
[]
[2]
[1]
[]
[]
[]
[1]
[3]
[0]
[2, 3]
[]
[]
[0]
[]
[0, 1, 3]
[3]
[]
[3]
[1]
[3]
[0]
[2]
[1]
[3]
[]
[]
[1]
[]
[0]
[2]
[1]
[]
[0]
[2]
[1]
[3]
[0]
[2]
[]
[]
[0]
[2]
[1]
[3]
[0]
[2]
[1]
[3]
[0

In [9]:
predictions=seqmot.predict(seqs, return_list=False)
for p in predictions: print p

0
0
1
0
1
0
1
1
1
1
0
1
1
0
1
0
1
1
1
1
1
1
1
1
0
1
1
1
0
0
1
0
0
1
1
0
1
0
1
1
0
0
0
1
0
1
1
0
0
0
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
0
0
0
0
0
1
1
0
1
1
1
1
1
1
1
0
0
0
1
1
1
0
1
1
0
1
1
0
0
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
0
0
1
0
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
0
1
1
0
1
1
1
1
0
0
1
1
0
1
1
0
1
1
0
0
0
1
0
0
1
1
0
0
0
1
1
1
1
0
0
1
0
1
1
0
1
1
1
1
1
1
1
0
0
1
0
1
1
1
0
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
0
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
1
0
1
1
0
1
1
1
1
1
1
1
0
1
1
0
1
0
0
1
1
1
1
1
1
1
1
1
0
0
0
1
0
1
0
1
1
0
1
1
1
1
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
0
1
0
1
1
0
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
1
0
1
0
1
0
1
0
0
1
1
1
1
1
1
1
1
0
1
1
1
0
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
0
0
1
0
1
1
1
1
1
1
0
1
1


In [10]:
predictions=seqmot.transform(seqs, return_match=True)
for p in predictions: print p

[[], [], [], []]
[[], [], [], []]
[[(62, 71)], [], [], [(3, 13)]]
[[], [], [], []]
[[], [(49, 59)], [], []]
[[], [], [], []]
[[(44, 54)], [], [], []]
[[(9, 19)], [], [], []]
[[], [(47, 56)], [], []]
[[(9, 19)], [], [], [(44, 54)]]
[[], [], [], []]
[[], [], [(22, 32)], []]
[[(56, 65)], [(70, 79)], [], []]
[[], [], [], []]
[[(45, 55)], [], [], []]
[[], [], [], []]
[[(61, 71)], [(46, 56)], [], []]
[[], [(54, 64)], [], []]
[[(43, 52)], [], [], []]
[[], [], [(45, 55)], []]
[[], [(46, 55)], [], []]
[[], [], [], [(45, 54)]]
[[(43, 53)], [], [(2, 12)], []]
[[], [], [(45, 55)], []]
[[], [], [], []]
[[], [], [(66, 75)], []]
[[(44, 53)], [], [], []]
[[], [], [(46, 55)], []]
[[], [], [], []]
[[], [], [], []]
[[(45, 54)], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], [(45, 54)]]
[[(48, 57)], [], [], []]
[[], [], [], []]
[[], [(47, 56)], [], []]
[[], [], [], []]
[[], [(0, 9)], [], []]
[[], [], [(44, 54)], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [], []]
[[], [], [(42, 51)], []]
[[

In [11]:
predictions=seqmot.transform(seqs, return_match=False)
for p in predictions: print p

[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[1, 1, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[1, 0, 1, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]
[0, 1, 0, 1]
[1, 1, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 1]
[0, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 0, 0]
[1, 0, 0, 0]
[1, 0, 0, 0]
[0, 1, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 1, 0, 0]
[0, 0, 0, 1]
[1, 0, 0, 0]
[0, 0, 0, 0]
[0, 1, 0, 0]
[0, 0, 0, 0]
[1, 0, 0, 0]
[0, 0, 1, 0]
[0, 0, 0, 0]

In [12]:
%%time
from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=150, verbosity=1, algorithm="MiniBatchKMeans", n_clusters=4)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

model induction: 22 secs
motives extraction: 617 motives 22 secs
motives clustering: 4 clusters 1 secs
After filtering: 0 secs
  # motives: 468
  # clusters: 4
motives models: 0 secs
0
UUUUUUUUUU 10
UUUUUUUUU 5
UUUUUUUUC 4
UUUUUAUUUU 3
UUUCUUUUUU 3
UCUUUUUUUU 3
UUUUUUUUUC 2
UUUUUUCUUU 2
UUUUGUUUUU 2
UUUGUUUUUU 2
UCUUUUUUUG 2
GUUUUUUUG 2
CUUUUUCUUU 2
AUAUUUUUUU 2
UUUUUUUUGG 1
UUUUUUUUG 1
UUUUUUUUCU 1
UUUUUUUUAU 1
UUUUUUUGUU 1
UUUUUUGUUU 1
UUUUUUCUUA 1
UUUUUUCUC 1
UUUUUUAUC 1
UUUUUCUUUU 1
UUUUUCUUU 1
UUUUUCUU 1
UUUUUAUUG 1
UUUUCUUUUU 1
UUUUCCUCUC 1
UUUUAUUUUU 1
UUUUAUUUUA 1
UUUUACGAGC 1
UUUUACCACA 1
UUUGUUUUUC 1
UUUCUCUUUU 1
UUUCUAUUU 1
UUUAUUUUUU 1
UUCUUUUUUU 1
UUCUUUUUUC 1
UUCUUUUCUG 1
UUCUUUUCU 1
UUCUUAUUUU 1
UCUUUUUAUA 1
UCUUUUGUUU 1
UCUCUUUUC 1
UCUAUCACA 1
UAUUUUUCUU 1
UAUCUCCUCA 1
GUUUUUUUUU 1
GUUUUUUUUG 1
GUUUUUUUUC 1
GUUUUUUUU 1
GUUCUUUUU 1
GUCUUUCUA 1
CUUUUUUUUU 1
CUUUUUUUUG 1
CUUUUUUUUC 1
CUUUUUUUUA 1
CUUUUUUUC 1
CUUUUUUCUG 1
CUUUUUCUU 1
CUUUUUAUCU 1
CUUUCUUUUU 1
CUCUUUUUUG 1
C

In [13]:
%%time
from eden.motif import SequenceMotif
seqmot = SequenceMotif(training_size=150, verbosity=1, algorithm="DBSCAN", eps=0.3, min_samples=3)
seqmot.fit(seqs)

for cluster_id in seqmot.motives_db:
    print cluster_id
    for count, motif in sorted(seqmot.motives_db[cluster_id], reverse=True):
        print motif, count

model induction: 23 secs
motives extraction: 609 motives 22 secs
motives clustering: 9 clusters 1 secs
After filtering: 0 secs
  # motives: 359
  # clusters: 8
motives models: 0 secs
0
AAAAAAAAAA 12
AAAAAAAAA 5
CAAAAAAAAU 4
AAAAAAAAAU 4
UAAAAAAAAU 3
GAAAAAAAU 3
GAAAAAAAAA 3
CAAAAUAAA 3
CAAAAAAAAC 3
CAAAAAAAAA 3
UAAUAAAAU 2
UAAAUAUAU 2
UAAAAGAAAG 2
UAAAAAAAU 2
UAAAAAAAAA 2
GACAAAAAAC 2
CAAAAAUAAG 2
CAAAAAAAA 2
ACACAAAAAA 2
AAAUAAAAAC 2
AAACAAAGAC 2
AAAAUAAAAA 2
AAAAAAAAU 2
UACAAAUAC 1
UAAUAACAAU 1
UAAUAACAAA 1
UAACAAUAA 1
UAAAAUAAA 1
UAAAAGAAU 1
UAAAAGAAA 1
UAAAAAUAC 1
UAAAAAUAAU 1
UAAAAAGAAU 1
UAAAAAAGAA 1
UAAAAAAAG 1
UAAAAAAAA 1
GUAACAAUAU 1
GAACAAAAAC 1
GAAAAAUAAU 1
GAAAAAGAA 1
GAAAAACAAC 1
GAAAAAAUAU 1
GAAAAAAAG 1
GAAAAAAAAG 1
CUAAUAAAAU 1
CCAAUAAUAU 1
CAUAAACAAU 1
CAGAAAAAC 1
CACAAAGAAU 1
CAAUAAUAAA 1
CAAUAAAAAU 1
CAAAUAAAAG 1
CAAAUAAAAC 1
CAAAGAAAU 1
CAAAAUAAU 1
CAAAACACAU 1
CAAAACAAU 1
CAAAACAAA 1
CAAAAAUAC 1
CAAAAACAAA 1
CAAAAAAUAG 1
CAAAAAAAU 1
CAAAAAAAG 1
CAAAAAAAC 1
AUAAACAAC